<a href="https://colab.research.google.com/github/federicoding/TrainingGeneral/blob/main/Debut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Train Python

In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/federicoding/TrainingGeneral/main/Transactions.csv'
transactions = pd.read_csv(filepath_or_buffer=url, sep=',', header=0, index_col="transaction_id")

In [4]:
transactions.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop


In [5]:
transactions.shape

(23053, 9)

Préparation des variables catégorielles et quantitatives

In [7]:
cat_vars = transactions[['cust_id', 'tran_date', 'prod_subcat_code', 'prod_cat_code', 'Store_type']]
num_vars = transactions[['Qty', 'Rate', 'Tax', 'total_amt']]
print(cat_vars.head())
num_vars.head()

                cust_id   tran_date  prod_subcat_code  prod_cat_code Store_type
transaction_id                                                                 
80712190438      270351  28-02-2014                 1              1     e-Shop
29258453508      270384  27-02-2014                 5              3     e-Shop
51750724947      273420  24-02-2014                 6              5   TeleShop
93274880719      271509  24-02-2014                11              6     e-Shop
51750724947      273420  23-02-2014                 6              5   TeleShop


,Qty,Rate,Tax,total_amt
transaction_id,,,,
80712190438,-5,-772,405.300,-4265.300
29258453508,-5,-1497,785.925,-8270.925
51750724947,-2,-791,166.110,-1748.110
93274880719,-3,-1363,429.345,-4518.345
51750724947,-2,-791,166.110,-1748.110


On split vite fait des données.

In [8]:
transactions_eshop = transactions.loc[transactions.Store_type=='e-Shop']
transactions_id_date = transactions_eshop[['cust_id','tran_date']]
transactions_id_date.head()

,cust_id,tran_date
transaction_id,,
80712190438,270351,28-02-2014
29258453508,270384,27-02-2014
93274880719,271509,24-02-2014
45649838090,273667,22-02-2014
50076728598,269014,21-02-2014


On regarde un client en particulier

In [9]:
transactions_client_268819 = transactions[transactions['cust_id'] == 268819]

total = 0

for amount in transactions_client_268819['total_amt']:
  total += amount

print(total)

14911.974999999999


**Stats de base**

In [11]:
transactions.describe()

,cust_id,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt
count,23053.000000,23053.000000,23053.000000,23053.000000,23053.000000,23053.000000,23053.000000
mean,271021.746497,6.149091,3.763632,2.432395,636.369713,248.667192,2107.308002
std,2431.692059,3.726372,1.677016,2.268406,622.363498,187.177773,2507.561264
min,266783.000000,1.000000,1.000000,-5.000000,-1499.000000,7.350000,-8270.925000
25%,268935.000000,3.000000,2.000000,1.000000,312.000000,98.280000,762.450000
50%,270980.000000,5.000000,4.000000,3.000000,710.000000,199.080000,1754.740000
75%,273114.000000,10.000000,5.000000,4.000000,1109.000000,365.715000,3569.150000
max,275265.000000,12.000000,6.000000,5.000000,1500.000000,787.500000,8287.500000


**Comptage** des valeurs différentes

In [12]:
transactions.Store_type.value_counts()

e-Shop            9311
MBR               4661
Flagship store    4577
TeleShop          4504
Name: Store_type, dtype: int64

In [13]:
transactions['Store_type'].value_counts()

e-Shop            9311
MBR               4661
Flagship store    4577
TeleShop          4504
Name: Store_type, dtype: int64

Questions sur des résultats du describe.
Soit on affiche direct le describe et on se met à lire.
Soit on demande les quantités, direct, c'est ce que je vais faire ici.

In [14]:
transactions.total_amt.mean(),transactions.Qty.max()

(2107.3080019953964, 5)

On peut bidouiller 'en avance de phase' les datas

In [15]:
transactions[transactions['total_amt'] > 0].describe()

,cust_id,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt
count,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000,20876.000000
mean,271027.141981,6.156639,3.761496,3.003497,785.927764,247.861232,2608.444389
std,2433.338935,3.722363,1.675859,1.420479,415.586498,186.644248,1964.208517
min,266783.000000,1.000000,1.000000,1.000000,70.000000,7.350000,77.350000
25%,268938.000000,3.000000,2.000000,2.000000,426.000000,97.938750,1030.688750
50%,271004.000000,5.000000,4.000000,3.000000,788.000000,198.555000,2089.555000
75%,273122.250000,10.000000,5.000000,4.000000,1148.000000,363.510000,3825.510000
max,275265.000000,12.000000,6.000000,5.000000,1500.000000,787.500000,8287.500000


In [16]:
#ou
transactions[transactions['total_amt'] > 0].total_amt.mean()

2608.444389250814

#Data cleaning rapide

In [3]:
#reboot?
url = 'https://raw.githubusercontent.com/federicoding/TrainingGeneral/main/Transactions.csv'
transactions = pd.read_csv(filepath_or_buffer=url, sep=',', header=0, index_col="transaction_id")

Recherche de **duplicates**

Avec **duplicated** qui ressort une Series de False&True si la ligne est un doublon.

Pour en avoir le **total** on en finit avec **.sum()**

In [19]:
print("Il y a : ",transactions.duplicated().sum(), "doublons dans le dataframe de transactions")

Il y a :  13 doublons dans le dataframe de transactions


Suppression des **doublons**

Avec **drop_duplicates** avec la syntaxe suivante:

drop_duplicates(subset, keep, inplace)

*   Paramètre **subset** indique la ou les colonnes à considérer pour identifier et supprimer les doublons. Par défaut, **subset = None**: on considère toutes les colonnes du DataFrame.
*   Paramètre keep indique quelle entrée garder:
    * **'first'**: on garde la **première** occurence
    * **'last'**: on garde la **dernière** occurence
    * **'False'**: on ne garde **aucune** des occurences
    * par défaut, **keep = 'first'**
  
*  Param **inplace** précise si on modifie **directement** le DataFrame (si **True**) ou si la méthode renvoie une **copie** du DataFrame. Si inplace=True la méthode est **irréversible**. Par défaut, inplace = False.

In [4]:
transactions = transactions.drop_duplicates(keep='first')
transactions

,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
...,...,...,...,...,...,...,...,...,...
94340757522,274550,25-01-2011,12,5,1,1264,132.720,1396.720,e-Shop
89780862956,270022,25-01-2011,4,1,1,677,71.085,748.085,e-Shop
85115299378,271020,25-01-2011,2,6,4,1052,441.840,4649.840,MBR


In [21]:
transactions.drop_duplicates(subset='prod_cat_code', keep='first')

,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
43134751727,268487,20-02-2014,3,2,-1,-611,64.155,-675.155,e-Shop
25963520987,274829,20-02-2014,4,4,3,502,158.130,1664.130,Flagship store


In [4]:
url = 'https://raw.githubusercontent.com/federicoding/TrainingGeneral/main/Transactions.csv'
transactions = pd.read_csv(filepath_or_buffer=url, sep=',', header=0, index_col="transaction_id")

transactions = transactions.drop_duplicates(keep = 'first')

In [5]:
import numpy as np

On va remplacer des modalités, convertir et renommer des colonnes

In [ ]:
#transactions = transactions.replace(['e-Shop', 'TeleShop', 'MBR', 'Flagship store', np.nan],
#                                   [1, 2, 3, 4, 0])
#print(transactions.head(10))
#
#new_types = {'Store_type'        : 'int',
#             'prod_sub_cat_code' : 'int'}
#
#transactions = transactions.astype(new_types)
#
#new_names = {'Store_type'  : 'store_type',
#new_names = { 'Qty'        : 'qty',
#              'Rate'       : 'rate',
#              'Tax'        : 'tax'}
#
#transactions = transactions.rename(new_names, axis = 1)
#
#transactions.head()

Bon, ça galère en dictionnary ><

In [6]:
transactions = transactions.replace(['e-Shop', 'TeleShop', 'MBR', 'Flagship store', np.nan],
                                   [1, 2, 3, 4, 0])

In [7]:
transactions['Store_type'] = transactions['Store_type'].astype('int')

In [8]:
transactions['prod_subcat_code'] = transactions['prod_subcat_code'].astype('int')

In [9]:
new_names = {'Store_type'  : 'store_type',
              'Qty'        : 'qty',
              'Rate'       : 'rate',
              'Tax'        : 'tax'}

transactions = transactions.rename(new_names, axis = 1)

transactions.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,1
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,1
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,2
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,1
51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,2


On va split la date (qui est en texte) en trois valeurs traitables (pas du texte quoi ^^)

Pour ça on crée 3 fonctions qui splittent et renvoient un n-ième élément, et ensuite on l'applique au jeu de données avec un **apply**

In [10]:
transactions = transactions[~transactions['tran_date'].str.contains(pat='/')]

**Nota Bene** belle galère car mon fichier n'est pas aussi clean, donc je dois enlever des trucs en plus avant que ça puisse passer...


In [34]:
transactions['tran_date'].isnull().values.any()

False

In [24]:
transactions['tran_date'].iloc[0].split('-')[2]

'2014'

In [11]:
def get_day(date) :
    return date.split("-")[0]



In [12]:
def get_month(date) :
    return date.split("-")[1]

In [13]:
def get_year(date) :
    return date.split("-")[1]

**ça marche une fois qu'on a filtré toutes les dates avec putain de /**

In [14]:
transactions['day'], transactions['month'], transactions['year'] = transactions.tran_date.apply(get_day), transactions.tran_date.apply(get_month), transactions.tran_date.apply(get_year)
transactions.head(10)

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,day,month,year
transaction_id,,,,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,1,28,02,02
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,1,27,02,02
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,2,24,02,02
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,1,24,02,02
51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,2,23,02,02
97439039119,272357,23-02-2014,8,3,-2,-824,173.040,-1821.040,2,23,02,02
45649838090,273667,22-02-2014,11,6,-1,-1450,152.250,-1602.250,1,22,02,02
22643667930,271489,22-02-2014,12,6,-1,-1225,128.625,-1353.625,2,22,02,02
79792372943,275108,22-02-2014,3,1,-3,-908,286.020,-3010.020,3,22,02,02


In [36]:
#days = transactions['tran_date'].apply(get_day)


In [ ]:
#months = transactions['tran_date'].apply(get_month)

In [ ]:
#years = transactions['tran_date'].apply(get_year)

Je perds patience mais il semble que certaines dates n'aient pas le bon format / longueur

In [41]:
transactions['tran_date'].sort_values( key = lambda x: x.str.len())

transaction_id
71286190189      5/9/2011
76929396378      8/4/2013
79507616107      8/4/2013
64276885615      8/4/2013
6539826915       8/4/2013
                  ...    
7199499669     19-11-2012
32511506570    19-11-2012
35525966352    19-11-2012
99959918143    19-11-2012
77960931771    25-01-2011
Name: tran_date, Length: 23040, dtype: object

OK y a des dates qui ont des **putain de / de mes deux**, forcément, en split sur **-** ben ça passe pas ><

Faudrait filtrer et tout et tout, mais pour l'instant, je passe...

Ou alors, petit détour vite fait...

In [44]:
y_a_un_slash = transactions['tran_date'].str.contains(pat = '/')
y_a_un_slash

transaction_id
71286190189     True
76929396378     True
79507616107     True
64276885615     True
6539826915      True
               ...  
7199499669     False
32511506570    False
35525966352    False
99959918143    False
77960931771    False
Name: tran_date, Length: 23040, dtype: bool

In [48]:
#transactions2 = transactions[~transactions['tran_date'].str.contains(pat='/')]

In [49]:
#transactions2.describe()

,cust_id,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
count,13918.000000,13918.000000,13918.000000,13918.000000,13918.000000,13918.000000,13918.000000,13918.000000
mean,271043.425277,6.122288,3.759520,2.434401,635.744504,249.997395,2114.047129,2.201466
std,2428.196652,3.732157,1.683648,2.274470,625.718711,187.731059,2521.095774,1.166313
min,266783.000000,1.000000,1.000000,-5.000000,-1499.000000,7.350000,-8270.925000,1.000000
25%,268969.000000,3.000000,2.000000,1.000000,310.000000,98.175000,762.450000,1.000000
50%,271027.000000,5.000000,4.000000,3.000000,713.000000,201.967500,1763.580000,2.000000
75%,273121.750000,10.000000,5.000000,4.000000,1112.750000,368.707500,3606.720000,3.000000
max,275265.000000,12.000000,6.000000,5.000000,1500.000000,787.500000,8287.500000,4.000000


Petite manip de row etc etc

In [15]:
transactions['prod_cat'] = transactions.astype('str').apply(lambda row: row['prod_cat_code']+'-'+row['prod_subcat_code'],
                                                            axis = 1)
print(transactions['prod_cat'])

transaction_id
80712190438     1-1
29258453508     3-5
51750724947     5-6
93274880719    6-11
51750724947     5-6
               ... 
94340757522    5-12
89780862956     1-4
85115299378     6-2
72870271171    5-11
77960931771    5-11
Name: prod_cat, Length: 13918, dtype: object


Vite fait les valeurs manquantes (*el classico*)

Avec primo une relance

In [24]:
url = 'https://raw.githubusercontent.com/federicoding/TrainingGeneral/main/Transactions.csv'
transactions = pd.read_csv(filepath_or_buffer=url, sep=',', header=0, index_col="transaction_id")

transactions = transactions.drop_duplicates(keep = 'first')

In [25]:
new_names = {'Store_type'  : 'store_type',
              'Qty'        : 'qty',
              'Rate'       : 'rate',
              'Tax'        : 'tax'}

transactions = transactions.rename(new_names, axis = 1)

transactions.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop


Detection des **colonnes** avec **au moins une NA**

In [26]:
transactions.isna().any(axis=0)

cust_id             False
tran_date           False
prod_subcat_code    False
prod_cat_code       False
qty                 False
rate                False
tax                 False
total_amt           False
store_type          False
dtype: bool

Detection des **lignes** avec **au moins une NA** 

In [27]:
transactions.isna().any(axis=1)

transaction_id
80712190438    False
29258453508    False
51750724947    False
93274880719    False
51750724947    False
               ...  
94340757522    False
89780862956    False
85115299378    False
72870271171    False
77960931771    False
Length: 23040, dtype: bool

Affichage des **entrées contenant des NA**

In [28]:
transactions[transactions.isna().any(axis=1)]

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,


**Comptage** du **nb de NA** dans une **colonne**

In [29]:
transactions.isnull().sum(axis=0)

cust_id             0
tran_date           0
prod_subcat_code    0
prod_cat_code       0
qty                 0
rate                0
tax                 0
total_amt           0
store_type          0
dtype: int64

**Comptage** du **nb de NA** par **ligne**

In [30]:
transactions.isnull().sum(axis=1)

transaction_id
80712190438    0
29258453508    0
51750724947    0
93274880719    0
51750724947    0
              ..
94340757522    0
89780862956    0
85115299378    0
72870271171    0
77960931771    0
Length: 23040, dtype: int64

In [31]:
transactions.isna()
print(transactions.isna().any(axis=0).sum()," colonnes contiennent des valeurs manquantes.")
print(transactions.isna().any(axis=1).sum()," lignes contiennent des valeurs manquantes.")
transactions.describe()

0  colonnes contiennent des valeurs manquantes.
0  lignes contiennent des valeurs manquantes.


,cust_id,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt
count,23040.000000,23040.000000,23040.000000,23040.000000,23040.000000,23040.000000,23040.000000
mean,271021.880252,6.148785,3.763498,2.435764,637.094965,248.677488,2109.865226
std,2431.573668,3.726197,1.677091,2.264326,621.727374,187.188311,2505.610295
min,266783.000000,1.000000,1.000000,-5.000000,-1499.000000,7.350000,-8270.925000
25%,268935.000000,3.000000,2.000000,1.000000,312.000000,98.280000,762.450000
50%,270980.500000,5.000000,4.000000,3.000000,710.000000,199.080000,1756.950000
75%,273114.250000,10.000000,5.000000,4.000000,1109.000000,365.767500,3570.255000
max,275265.000000,12.000000,6.000000,5.000000,1500.000000,787.500000,8287.500000


**Remplacement** des **NA**

Méthode **fillna()**

Pour les colonnes de **type numérique**:

*   Par une valeur constante: df.fillna(0)
*   Par la moyenne (*e.g.*): df.fillna(df.mean())
*   On peut aussi par la médiane (utiliser **median**) ou par le min/max (utiliser **min**/**max**)

Pour les colonnes de **type catégorielle**

*   Par le **mode**, *i.e.* la modalité la plus fréquente (utiliser **mode**)
*   Par une constante ou catégorie arbitraire: 0, -1

Faire attention à bien travailler sur la colonne cible, sinon c'est le bordel.

**Suppression** des valeurs manquantes **NA**

Méthode **dropna**

Syntaxe:   dropna(axis, how, subset,..)

*   Paramètre **axis** si on doit supprimer les lignes ou les colonnes (**0** pour les lignes, **1** pour les colonnes).
*   Paramètre **how** précise la méthodologie de suppression:
      *  **how = 'any'**: on supprime la ligne (ou colonne) si elle contient **au moins une* valeur manquante.
      *  **how = 'all'** on supprime la ligne (ou colonne) que si elle ne contient **que** des valeurs manquantes.

*   Paramètre **subset** permet de préciser les colonnes/lignes sur lesquelles on effectue la recherche de valeurs manquantes.

*e.g.* Suppression des lignes ayant des valeurs manquantes dans les colonnes suivantes 'col1', 'col3' et 'col5':

df.dropna(axis = 0, how = 'any', subset = ['col1','col3', 'col5'])

#Préprocessing

Généralement **4 opérations**:
*  **filtrer**
*  **unir**
*  **ordonner**
*  **grouper**

##Filtrer

Les opérations suivantes sont disponibles:
*  l'opérateur 'et': **&**
*  l'opérateur 'ou': **|**
*  l'opérateur 'non': **-**

**NB:** s'il faut vérifier **simultanément** des conditions, alors les isoler chacune dans sa paranthèse, l'isolant de **l'opérateur**.

Exemple de deux conditions remplies simultanément:

print(df[(df['année'] == 1953) **&** (df['surface'] >75)])

Remplacer **&** par **|** si l'on veut un *ou* logique, *i.e.* **au moins une** condition vérifiée.

In [32]:
# Création de e_shop et teleshop
teleshop = transactions[(transactions['store_type'] == 'TeleShop') & (transactions['total_amt'] > 5000)]

e_shop = transactions[(transactions['store_type'] == 'e-Shop') & (transactions['total_amt'] > 5000)]

# Dénombrement des lignes des 2 DataFrame avec la fonction len. D'autres solutions existent.
print('Nombre de transactions à plus de 5000€ pour le e-shop :', len(e_shop['total_amt']))
print('Nombre de transactions à plus de 5000€ pour le TeleShop :', len(teleshop['total_amt']))

Nombre de transactions à plus de 5000€ pour le e-shop : 1190
Nombre de transactions à plus de 5000€ pour le TeleShop : 536


On s'attaque maintenant aux autres fichiers:

In [33]:
url = 'https://raw.githubusercontent.com/federicoding/TrainingGeneral/main/Customer.csv'
customer = pd.read_csv(filepath_or_buffer=url, sep=',')

url = 'https://raw.githubusercontent.com/federicoding/TrainingGeneral/main/prod_cat_info.csv'
prod_cat_info = pd.read_csv(filepath_or_buffer=url, sep=',')

Remplacement du **Gender** et du **city_code** par leur mode

In [35]:
customer['Gender'] = customer['Gender'].fillna(customer['Gender'].mode()[0])
customer['city_code'] = customer['city_code'].fillna(customer['city_code'].mode()[0])

##Unir

Avec les fonctions **concat** et **merge**

**concat** permet de concaténer plusieurs DataFrames, c'est à dire les juxtaposer horizontalement ou verticalement.

`pd.concat(obj, axis..`

*   Le paramètre `obj` contient la liste des `DataFrames` à concaténer
*   Le paramètre `axis` précise si l'on doit concaténer verticalement (`axis = 0`) [par *en dessous* quoi] ou horizontalement (`axis = 1`) [collé *côte à côte* / *rajout de colonnes*]. 

Méthode **merge** 

Utile si les deux `DataFrames` ont une colonne en commun, via:

`merge(right, on how, ...)`

*   Le paramètre **right** est le `DataFrame` à fusionner avec celui qui appelle la méthode
*   Le paramètre **on** est le nom des colonnes des `DataFrame` qui serviront de référence pour la fusion. Elles doivent être **communes** aux deux `DataFrames`
*   Le paramètre **how** permet de choisir le **type de jointure** à effectuer. Les valeurs de ce paramètre sont basées sur les jointures de la syntaxe SQL.

**how** peut prendre 4 valeurs (`'inner','outer,'left','right'`)

Voir les images (stockées sur le DD local malheureusement ^^).

Bref, en gros on fait souvent du `left`.

NB: on peut faire ça *proprement* en changeant les noms des colonnes avant et tout et tout

In [36]:
#genre:
# On renomme la colonne 'customer_Id' en 'cust_id' pour faire la fusion
customer = customer.rename(columns = {'customer_Id':'cust_id'})

# Jointure à gauche entre transactions et customer sur la colonne 'cust_id'
fusion = transactions.merge(right = customer, on = 'cust_id', how = 'left')

# La fusion n'a produit aucun NaN
fusion.isna().sum()

# Les colonnes DOB, Gender, city_code ont bien été ajoutées à transactions
fusion.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,DOB,Gender,city_code
0,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop,26-09-1981,M,5.0
1,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop,11-05-1973,F,8.0
2,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop,27-07-1992,M,8.0
3,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop,08-06-1981,M,3.0
4,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop,27-07-1992,M,8.0


Ou un peu plus salement, mais rapidement

In [ ]:
#genre:
#merge "prod_concat" and "customer" table and create the final table "customer_final"
#customer_final = pd.merge(left=prod_concat, right=customer,right_on="customer_Id", left_on="cust_id", how="left")

La **fusion** s'est bien passée, mais **l'index** à sauté. Il faut le refaire. Avec la méthode **set_index**.

Il faut donner à manger à `set_index()` soit:
*   le **nom** d'une colonne à utiliser comme indexation
*  un `array` Numpy ou une `Series` pandas avec le même nombre de lignes que le `DataFrame` appelant la méthode.

In [37]:
new_index = transactions.index
fusion = fusion.set_index(new_index)
fusion.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,DOB,Gender,city_code
transaction_id,,,,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop,26-09-1981,M,5.0
29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop,11-05-1973,F,8.0
51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop,27-07-1992,M,8.0
93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop,08-06-1981,M,3.0
51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop,27-07-1992,M,8.0


##Trier et Ordonner

Deux méthodes: **sort_values** et **sort_index**

**sort_values** permet de trier les lignes d'une `DataFrame` selon les valeurs d'une ou de plusieurs colonnes, syntaxe:

`sort_values(by, ascending,...)`

*   le paramètre `by` permet de préciser sur quelle(s) colonne(s) le tri est effectué
*   le paramètre `ascending` est un booléen (`True` ou `False`) déterminant l'odre croissant/décroissant du tri. Par défaut ce paramètre vaut `True`

On peut trier par *tiroirs* *i.e.* un premier tri global, par exemple par une valeur numérique, et ensuite par ordre alphabétique sur une deuxième colonne (sans toucher au premier tri).

**sort_index** permet de trier par l'index.

Petit exercice de style avec des `DataFrames` de bateaux

In [38]:
# Definition des dictionnaires
data_bateaux = {'nom_bateau'  : ['Julia', 'Siren', 'Sea Sons', 'Hercules', 'Cesar', 'Minerva'], 
                'couleur'    : ['bleu', 'vert', 'rouge', 'bleu', 'jaune', 'vert'],
                'numero_reservation': [2, 3, 6, 1, 4, 5],
                'nombre_reservations': [34, 10, 20, 41, 12, 16]}

data_clients = {'id_client' : [91, 154, 124, 320, 87, 22], 
                'nom_client'        : ['Marie', 'Anna', 'Yann', 'Lea', 'Marc', 'Yassine'],
                'id_reservation': [1, 2, 3, 7, 9, 5]}

# Creation des DataFrames
bateaux = pd.DataFrame(data_bateaux)
clients = pd.DataFrame(data_clients)

In [39]:
#on renome la colonne
bateaux = bateaux.rename(columns={'numero_reservation':'id_reservation'})

#on joint bateaux et clients
bateaux_clients = bateaux.merge(right = clients, on='id_reservation', how='left')

#on indexe avec la colonne 'nom_bateau'
bateaux_clients = bateaux_clients.set_index('nom_bateau')

On cherche qui a réservé les bateaux `Julia` et `Siren`

In [41]:
print("Le client qui a réservé 'Julia' est:", bateaux_clients.loc['Julia','nom_client'])
print("Le client qui a réservé 'Siren' est:", bateaux_clients.loc['Siren', 'nom_client'])

Le client qui a réservé 'Julia' est: Anna
Le client qui a réservé 'Siren' est: Yann


Avec la méthode **`isna`** appliquée à la colonne `nom_client` déterminer les bateaux non réservés

In [42]:
bateaux_non_reserves = bateaux_clients[bateaux_clients['nom_client'].isna()]
print("Les bateaux qui n'ont pas été réservés sont:", [bateau for bateau in bateaux_non_reserves.index])

Les bateaux qui n'ont pas été réservés sont: ['Sea Sons', 'Cesar']


Avec **sort_values** déterminer le nom du client qui a réservé le bateau bleu ayant le plus de réservations à son actif.

In [48]:
bateaux_clients = bateaux_clients.sort_values(by=['couleur','nombre_reservations'], ascending=[True,False])

print(bateaux_clients)

print("Le bateau bleu le plus réservé l'est par:",bateaux_clients.nom_client[0])

           couleur  id_reservation  nombre_reservations  id_client nom_client
nom_bateau                                                                   
Hercules      bleu               1                   41       91.0      Marie
Julia         bleu               2                   34      154.0       Anna
Cesar        jaune               4                   12        NaN        NaN
Sea Sons     rouge               6                   20        NaN        NaN
Minerva       vert               5                   16       22.0    Yassine
Siren         vert               3                   10      124.0       Yann
Le bateau bleu le plus réservé l'est par: Marie
